# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [1]:
# Import the data
import pandas as pd

df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [2]:
# Total number of people who survived/didn't survive
df.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [3]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
dummy_dataframe = pd.get_dummies(df[relevant_columns], drop_first=True, dtype=float)

dummy_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Age         714 non-null    float64
 2   SibSp       891 non-null    int64  
 3   Fare        891 non-null    float64
 4   Survived    891 non-null    int64  
 5   Sex_male    891 non-null    float64
 6   Embarked_Q  891 non-null    float64
 7   Embarked_S  891 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 55.8 KB


Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [4]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
dummy_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      714 non-null    int64  
 1   Age         714 non-null    float64
 2   SibSp       714 non-null    int64  
 3   Fare        714 non-null    float64
 4   Survived    714 non-null    int64  
 5   Sex_male    714 non-null    float64
 6   Embarked_Q  714 non-null    float64
 7   Embarked_S  714 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 50.2 KB


Finally, assign the independent variables to `X` and the target variable to `y`: 

In [5]:
# Split the data into X and y
y = dummy_dataframe[['Survived']]
X = dummy_dataframe.drop('Survived', axis=1)

## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [6]:
# Build a logistic regression model using statsmodels
import statsmodels.api as sm

# Create intercept term required for sm.Logit, see documentation for more information
X = sm.add_constant(X)

# Fit model
logit_model = sm.Logit(y, X)

# Get results of the fit
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.443267
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [7]:
# Summary table
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      706
Method:                           MLE   Df Model:                            7
Date:                Sat, 04 Apr 2020   Pseudo R-squ.:                  0.3437
Time:                        12:44:01   Log-Likelihood:                -316.49
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.103e-67
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6503      0.633      8.921      0.000       4.409       6.892
Pclass        -1.2118      0.163     -7.433      0.000      -1.531      -0.892
Age           -0.0431      0.008     -5.250      0.000      -0.059      -0.027
SibSp         -0.3806      0.125     -3.048      0.002      -0.625      -0.136
Fare           0.0012      0.002      0.474      0.636      -0.004       0.006
Sex_male      -2.6236      0.217    -12.081      0.000      -3.049      -2.198
Embarked_Q    -0.8260      0.598     -1.381      0.167      -1.999       0.347
Embarked_S    -0.4130      0.269     -1.533      0.125      -0.941       0.115
==============================================================================
"""

In [8]:
import numpy as np

np.exp(result.params)

const         284.370886
Pclass          0.297670
Age             0.957778
SibSp           0.683485
Fare            1.001153
Sex_male        0.072540
Embarked_Q      0.437799
Embarked_S      0.661656
dtype: float64

In [9]:
# Your comments here
from IPython.core.display import HTML, Markdown

good_pval_feats = []
npw_series_index = np.where(result.pvalues < 0.05)
display(HTML(f"<h3>Good p-values (< 0.05):</h3>"))
for idx, p_val in enumerate(result.pvalues[npw_series_index[0]]):
    good_pval_feat = result.pvalues[npw_series_index[0]].index[idx]
    print(f"{good_pval_feat}: {p_val}")
    good_pval_feats.append(good_pval_feat)

npw_series_index = np.where(result.pvalues >= 0.05)
display(HTML(f"<p><br><h3>Bad p-values (>= 0.05):</h3>"))
for idx, p_val in enumerate(result.pvalues[npw_series_index[0]]):
    bad_pval_feat = result.pvalues[npw_series_index[0]].index[idx]
    print(f"{bad_pval_feat}: {p_val}")

const: 4.638711472530189e-19
Pclass: 1.0620275848571316e-13
Age: 1.5171522101176506e-07
SibSp: 0.002305214424653843
Sex_male: 1.3279678233575505e-33


Fare: 0.6357191505326099
Embarked_Q: 0.16736727353695657
Embarked_S: 0.12539366144021524


## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [10]:
# Your code here

# Create intercept term required for sm.Logit, see documentation for more information
X = sm.add_constant(X[good_pval_feats])

# Fit model
logit_model = sm.Logit(y, X[good_pval_feats])

# Get results of the fit
result = logit_model.fit()

result.summary()

Optimization terminated successfully.
         Current function value: 0.445882
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      709
Method:                           MLE   Df Model:                            4
Date:                Sat, 04 Apr 2020   Pseudo R-squ.:                  0.3399
Time:                        12:44:01   Log-Likelihood:                -318.36
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.089e-69
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6008      0.543     10.306      0.000       4.536       6.666
Pclass        -1.3174      0.141     -9.350      0.000      -1.594      -1.041
Age           -0.0444      0.008     -5.442      0.000      -0.060      -0.028
SibSp         -0.3761      0.121     -3.106      0.002      -0.613      -0.139
Sex_male      -2.6235      0.215    -12.229      0.000      -3.044      -2.203
==============================================================================
"""

In [11]:
np.exp(result.params)

const       270.655330
Pclass        0.267831
Age           0.956586
SibSp         0.686521
Sex_male      0.072550
dtype: float64

In [12]:
# Your comments here

# not much has changed
# Pseudo R-squ. dropped from 0.3437 to 0.3399
# LLR p-value improved from 1.103e-67 to 1.089e-69
# sex (male) has the largest abs coeff; surprisingly, Pclass has a large abs coeff as well

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!